In [ ]:
!pip install ultralytics --quiet
!pip install roboflow --quiet

In [2]:
from torchvision.transforms.functional import adjust_hue
from ultralytics import YOLO, settings
from torchvision import transforms
from roboflow import Roboflow
from PIL import Image
import shutil
import torch
import os
import re

In [ ]:
def hue_degree_to_hue_value():
    return (torch.rand(1).item() * 60 - 30) / 360.0  # Random hue between -30 and +30 degrees

def augment_image(pil_img, name):
    images = [pil_img.copy()]  # Start with the original image

    # Apply a noticeable hue shift
    hue_adjustment = hue_degree_to_hue_value()
    images.append(adjust_hue(pil_img.copy(), hue_adjustment))

    # Apply a more pronounced saturation change
    images.append(transforms.ColorJitter(saturation=0.4)(pil_img.copy()))

    # Apply a more noticeable brightness change
    images.append(transforms.ColorJitter(brightness=0.3)(pil_img.copy()))

    # Apply a stronger contrast change
    images.append(transforms.ColorJitter(contrast=0.4)(pil_img.copy()))

    names = [name] + [name.replace('.jpg', '_hue.jpg'), name.replace('.jpg', '_sat.jpg'), name.replace('.jpg', '_bright.jpg'), name.replace('.jpg', '_cont.jpg')]
    return images, names

def augment_images(image_paths):
    augmented_batch = []
    for img_path in image_paths:
        pil_img = Image.open(img_path)  # Open the image file
        augmented_batch.append((augment_image(pil_img, os.path.basename(img_path))))  # Apply augmentation and extend the batch list
    return augmented_batch

def get_training_number():
  try:
    runs_dir = os.listdir(f'./runs/{task}')
    max = 0
    last_train = ''

    for dir in runs_dir:
      if 'train' in dir:
        numbers = re.findall(r'\d+', dir)
        if len(numbers) > 0:
          if int(numbers[0]) > max:
            max = int(numbers[0])

    if max == 0:
      last_train = ''
    else:
      last_train = max

    print(last_train)
    return last_train

  except:
    print("Error, check if the directory exists!")

# download dataset
from roboflow import Roboflow
rf = Roboflow(api_key="x5pZ44ydkd9As40Mglzv")
project = rf.workspace("ntnuscaledetection").project("ntnu_herbarium_segmentation_v2")
version = project.version(8)
dataset = version.download("yolov8")

###########################
## FOR DATA-AUGMENTATION ##
###########################

# # get paths
# path = os.path.join(dataset.location, 'train', 'images')
# label_path = os.path.join(dataset.location, 'train', 'labels')

# image_paths = [os.path.join(path, i) for i in os.listdir(path) if i.endswith(('.png', '.jpg', '.jpeg'))]

# # augment images
# augmented_images = augment_images(image_paths)

# # save images and labels
# for idx, item in enumerate(augmented_images):
#     original_name = item[1][0].replace('.jpg', '.txt')
#     original_label = os.path.join(label_path, original_name)
#     # copy original label with new names (hue, sat, bright, cont)
#     for name in item[1][1:]:
#         new_label = os.path.join(label_path, name.replace('.jpg', '.txt'))
#         shutil.copy(original_label, new_label)

#     # save images
#     for name, image in zip(item[1], item[0]):
#         image.save(os.path.join(path, name))

In [ ]:
import shutil
import yaml

if not os.path.exists('./datasets'):
  os.makedirs('./datasets')

try:
  shutil.move(f'{dataset.location}', './datasets')
except Exception as e:
  print(e)

pj_name = project.name.replace(' ', '-')
print(pj_name)
data_yaml_path = f'/content/datasets/{pj_name}-{version.version}/data.yaml'

with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)

data_yaml['path'] = f'{pj_name}-{version.version}'
data_yaml['train'] = f'train/images'
data_yaml['val'] = f'valid/images'
data_yaml['test'] = f'test/images'

with open(data_yaml_path, 'w') as file:
    yaml.dump(data_yaml, file, default_flow_style=False)

task = 'segment'

In [ ]:
!yolo ${task} train data=/content/datasets/{pj_name}-{version.version}/data.yaml model=yolov8m-seg epochs=100 imgsz=640 show_labels=False translate=0.0 scale=0.0 erasing=0.0 mosaic=0.0 cos_lr=True